<a href="https://colab.research.google.com/github/plaban1981/Hugging_Face_transformers_topics/blob/main/mosaicml_mpt_7b_storywriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -qU -e git+https://github.com/samhavens/just-triton-flash.git#egg=flash_attn 

  Preparing metadata (setup.py) ... done


In [2]:
!pip install -qU  einops transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 98.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00


## Define the Pipeline

In [4]:
from typing import Any, Dict, Tuple
import warnings

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#
import datetime
import os
from threading import Event, Thread
from uuid import uuid4

import requests
import torch
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

In [6]:
from getpass import getpass
import os
#
os.environ["HF_TOKEN"] = getpass()
# Configuration
HF_TOKEN = os.getenv("HF_TOKEN", None)

··········


In [5]:
class TextGenerationPipeline:
    def __init__(
        self,
        model_name,
        torch_dtype=torch.bfloat16,
        trust_remote_code=True,
        use_auth_token=None,
    ) -> None:
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch_dtype,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
            max_seq_len=10240, 
        )

        tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=trust_remote_code,
            use_auth_token=use_auth_token,
        )
        if tokenizer.pad_token_id is None:
            warnings.warn(
                "pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id."
            )
            tokenizer.pad_token = tokenizer.eos_token
        tokenizer.padding_side = "left"
        self.tokenizer = tokenizer

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.eval()
        self.model.to(device=device, dtype=torch_dtype)

        self.generate_kwargs = {
            "temperature": 0.8,
            "top_p": 0.95,
            "top_k": 50,
            "max_new_tokens": 512,
            "use_cache": True,
            "do_sample": True,
            "eos_token_id": self.tokenizer.eos_token_id,
            "pad_token_id": self.tokenizer.pad_token_id,
            "repetition_penalty": 1.02,  # 1.0 means no penalty, > 1.0 means penalty, 1.2 from CTRL paper
        }

    def __call__(
        self, instruction: str, **generate_kwargs: Dict[str, Any]
    ) -> Tuple[str, str, float]:
        input_ids = self.tokenizer(instruction, return_tensors="pt").input_ids
        input_ids = input_ids.to(self.model.device)
        gkw = {**self.generate_kwargs, **generate_kwargs}
        with torch.no_grad():
            output_ids = self.model.generate(input_ids, **gkw)
        # # Slice the output_ids tensor to get only new tokens
        # new_tokens = output_ids[0, len(input_ids[0]) :]
        output_tokens = output_ids[0]
        output_text = self.tokenizer.decode(output_tokens, skip_special_tokens=True)
        return output_text


In [7]:
# Define a custom stopping criteria
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

In [8]:
# Initialize the model and tokenizer
generate = TextGenerationPipeline(
    "mosaicml/mpt-7b-storywriter",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    use_auth_token=HF_TOKEN,
)
stop_token_ids = generate.tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- configuration_mpt.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- hf_prefixlm_converter.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- adapt_tokenizer.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- norm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- attention.py
- norm.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- blocks.py
- attention.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- param_init_fns.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/mosaicml/mpt-7b-storywriter:
- modeling_mpt.py
- hf_prefixlm_converter.py
- adapt_tokenizer.py
- blocks.py
- param_init_fns.py
- meta_init_context.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-storywriter/6825a30eda8e732334fe2884a0f433f5abff9545/attention.py:157: UserWarning: Using `attn_impl: torch`. If your model does not use `alibi` or `prefix_lm` we recommend using `attn_impl: flash` otherwise we recommend using `attn_impl: triton`.
  warnings.warn('Using `attn_impl: torch`. If your model does not use `alibi` or ' + '`prefix_lm` we recommend using `attn_impl: flash` otherwise ' + 'we recommend using `attn_impl: triton`.')


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<ipython-input-5-d0267459f661>:23: UserWarning: pad_token_id is not set for the tokenizer. Using eos_token_id as pad_token_id.
  warnings.warn(


In [9]:
def generate_results(instruction):
  print(f"Prompt \n:{instruction}")
  tokenized_input = generate.tokenizer(instruction, return_tensors="pt")
  #
  stop = StopOnTokens()
  #
  outputs = generate.model.generate(input_ids=tokenized_input['input_ids'].to('cuda:0'), 
                            max_new_tokens=200, 
                            temperature =0.1,
                            top_k=5, 
                            top_p=0.95,
                            stopping_criteria =StoppingCriteriaList([stop]))
  #
  answer = generate.tokenizer.batch_decode(outputs, skip_special_tokens=True)
  print(answer[0].rstrip())
  return answer[0].rstrip()

In [10]:
instruction = """# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. """


In [11]:
content = generate_results(instruction)
print(content)

Prompt 
:# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks and scrapes of others in their tents, who, like me, found it simplest to just sleep in their armor. It ached, but everything ached out here. It was war, after all. I scanned my tent. For all its comforts, I sought but one: my trusty sword. It was a fine example of a forgotten style. 
# My Finest Hour
## Chapter 1
*
The sun rose that morning just like any other morning. Bright, obtrusive, and dangerous. Night seemed like the only protection I had anymore, but I knew even that was a just a lie I told myself so I could rest a little easier.
I wasn't the only one to stir as the morning sun crested the horizon. I could hear the clanks 